# Introdução:

Análise de Dataset: Letras de Músicas

Este notebook compõe uma série de notebooks que tem como objetivo explorar e aplicar técnicas de NLP sobre o dataset `song_lyrics.csv`, conforme solicitado na prova prática para o CAEd.


# Configuração

In [ ]:
# Bibliotecas necessárias
import pandas as pd
import numpy as np
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Tarefa (b)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Baixando Dataset do Drive com gdown e extraindo zip
Link do Dataset: https://drive.google.com/file/d/13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c/view?usp=sharing

In [ ]:
# Baixar com gdown
!gdown 13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c

# Descompactar
with zipfile.ZipFile('dataset - letras.zip', 'r') as zip_ref:
    zip_ref.extractall('letras_dataset')

# Verificar arquivos
!ls letras_dataset

Downloading...
From (original): https://drive.google.com/uc?id=13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c
From (redirected): https://drive.google.com/uc?id=13T_SXgmSh9qGS2MJXZqQL6BQjfMs4S3c&confirm=t&uuid=28400186-a8ea-4658-a7a2-28a4a1e49364
To: /content/dataset - letras.zip
100% 3.26G/3.26G [00:39<00:00, 81.9MB/s]
song_lyrics.csv


# Leituras

Caso a `amostra_song_lyrics10.csv` já exista e esteja incluída no ambiente. Basta pular para a etapa de pré-processamento. A etapa já carrega a base de forma processada, evitando passos extras.

## Opção 1: Amostra criada anteriormente

In [ ]:
# Deve-se subir a amostra para o ambiente do colab. O nome do arquivo deve estar na variável path entre aspas
path = 'amostra_song_lyrics10.csv'
df = pd.read_csv(path)
df.head()

,title,tag,artist,year,views,features,lyrics,language
0,Ducky,rap,North Star (USA),2004,143,"{""Meko the Pharaoh"",Christbearer}","""Oh you'll love her"" (repeats all throughout)\...",en
1,The Heist Revisited,rap,Big L,2000,5204,{},"[Big L]\nYeah, yeah\nYeah-yeah, yeah-yeah-yeah...",en
2,Chinese New Year,rap,Clipse,2006,22502,"{""Roscoe P. Coldchain""}",[Chorus: Pusha T & Pharrell]\nI'm at your door...,en
3,Pop That Cannon,rap,Cassidy,2004,677,"{""Styles P"",""Swizz Beatz""}","[Intro - Swizz Beatz]\nAooow! banger, let's go...",en
4,Been This Way,rap,Scribe,2003,1014,{},[Intro]\nEvery MC in this industry wants one t...,en


## Opção 2: Lendo com chuncksize e criando amostra

In [ ]:
# Caminho do CSV
csv_path = 'letras_dataset/song_lyrics.csv'

# Colunas desejadas
usecols = ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'language']

# Parâmetros
chunksize = 50_000  # número de linhas por chunk
sample_frac = 0.1   # fração da amostra por chunk (10%)

# Lista para armazenar amostras
samples = []

# Leitura por chunks
for chunk in pd.read_csv(csv_path, usecols=usecols, chunksize=chunksize):
  sample = chunk.sample(frac=sample_frac, random_state=42)
  samples.append(sample)

# Concatenar todos os pedaços amostrados
df = pd.concat(samples).reset_index(drop=True)

# Visualizar forma e colunas
print(f"Total de linhas na amostra: {df.shape[0]}")
df.head()

# Salva a amostra reduzida
df.to_csv('amostra_song_lyrics10.csv', index=False)
print("Arquivo salvo como amostra_song_lyrics10.csv")

Total de linhas na amostra: 513486
Arquivo salvo como amostra_song_lyrics.csv


# Pré-processamento

## Remoção de Livros e Poemas

In [ ]:
# Configurações
path = 'amostra_song_lyrics10.csv'
chunksize = 10000
limite_palavras_livro = 2000

# Contadores
total_entradas = 0
removidos_livros = 0
removidos_poemas = 0
exemplos_livros = []
exemplos_poemas = []
entradas_mantidas = []

# Função para detectar poemas
def analisar_estrutura(letra):
  if not isinstance(letra, str) or not letra.strip():
    return False

  linhas = [linha.strip() for linha in letra.split('\n') if linha.strip()]
  if len(linhas) < 8:
    return False

  total_linhas = len(linhas)
  palavras = ' '.join(linhas).split()
  diversidade = len(set(palavras)) / len(palavras) if palavras else 0
  media_palavras = len(palavras) / total_linhas
  linhas_repetidas = total_linhas - len(set(linhas))
  proporcao_repetidas = linhas_repetidas / total_linhas
  linhas_curtas = [linha for linha in linhas if len(linha.split()) < 7]
  proporcao_curtas = len(linhas_curtas) / total_linhas

  # Filtros básicos
  if media_palavras < 4 or diversidade < 0.3:
      return False

  # Score poético
  score_poetico = proporcao_curtas * (1 - proporcao_repetidas)

  return score_poetico >= 0.6

# Leitura por partes
for chunk in pd.read_csv(path, usecols=['title', 'tag', 'artist', 'year', 'views', 'lyrics', 'language'], chunksize=chunksize):
  chunk.dropna(subset=['lyrics'], inplace=True)
  total_entradas += len(chunk)

  # Detectar livros
  chunk['len_lyrics'] = chunk['lyrics'].str.split().apply(len)
  filtro_livros = chunk['len_lyrics'] > limite_palavras_livro
  livros = chunk[filtro_livros]
  if len(exemplos_livros) < 2:
    exemplos_livros.extend(livros[['title', 'artist', 'len_lyrics', 'lyrics']].head(2).to_dict('records'))
  removidos_livros += len(livros)

  # Restante
  chunk = chunk[~filtro_livros]

  # Detectar poemas nas tags relevantes
  chunk.loc[:, 'tag'] = chunk['tag'].astype(str).str.lower()
  filtro_tag_poema = chunk['tag'].isin(['misc', 'other'])

  poemas_idx = []
  for idx, row in chunk[filtro_tag_poema].iterrows():
    if analisar_estrutura(row['lyrics']):
      poemas_idx.append(idx)
      if len(exemplos_poemas) < 2:
        exemplos_poemas.append({
          'title': row['title'],
          'artist': row['artist'],
          'score_poetico': '≈ estrutural',
          'lyrics': row['lyrics']
        })

  removidos_poemas += len(poemas_idx)
  chunk = chunk.drop(poemas_idx)

  # (opcional) salvar para uso posterior
  entradas_mantidas.append(chunk)

# Concatenar final se desejar
df = pd.concat(entradas_mantidas, ignore_index=True)

# Relatório
print("Relatório de Limpeza:")
print(f"Total de entradas analisadas: {total_entradas}")
print(f"Entradas removidas (livros): {removidos_livros}")
print(f"Entradas removidas (poemas): {removidos_poemas}")
print(f"Total removido: {removidos_livros + removidos_poemas}")
print(f"Total final mantido: {len(df)}")

print("\nExemplos removidos (livros):")
for e in exemplos_livros:
  print(f" - {e['title']} ({e['artist']}) | {e['len_lyrics']} palavras")
  print(f"Trecho:\n{e['lyrics'][:300]}\n")

print("\nExemplos removidos (poemas):")
for e in exemplos_poemas:
  print(f" - {e['title']} ({e['artist']})")
  print(f"Trecho:\n{e['lyrics'][:300]}\n")

Relatório de Limpeza:
Total de entradas analisadas: 513486
Entradas removidas (livros): 2488
Entradas removidas (poemas): 2960
Total removido: 5448
Total final mantido: 508038

Exemplos removidos (livros):
 - Paradise Lost Book 9 (John Milton) | 9030 palavras
Trecho:
No more of talk where God or Angel guest
With Man, as with his friend, familiar us'd
To sit indulgent, and with him partake
Rural repast; permitting him the while
Venial discourse unblam'd. I now must change
Those notes to tragick; distrust, and breach
Disloyal on the part of Man, revolt
And disobed

 - Rappers Delight (Sugarhill Gang) | 3195 palavras
Trecho:
[Chorus: Wonder Mike]
I said a hip-hop, the hippie, the hippie
To the hip, hip-hop and you don't stop the rockin'
To the bang-bang boogie, say up jump the boogie
To the rhythm of the boogie, the beat

[Verse 1: Wonder Mike]
Now, what you hear is not a test, I'm rapping to the beat
And me, the groove


Exemplos removidos (poemas):
 - The Tyger (William Blake)
Trecho:
T

## Limpeza no campo lyrics

In [ ]:
stop_words = set(stopwords.words('english'))

def clean_lyrics(text):
  text = text.lower()
  text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # remove URLs
  text = re.sub(r'\n+', ' ', text)
  text = re.sub(r'[^a-zA-Z ]', '', text)       # remove pontuação
  text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)  # Remove letras únicas isoladas
  text = re.sub(r'\s+', ' ', text).strip()     # Remove espaços múltiplos e bordas

  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
  return ' '.join(tokens)

df = df[df['language'] == 'en'].copy()
print(f"Total de letras em inglês: {df.shape[0]}")
df['lyrics'] = df['lyrics'].apply(clean_lyrics)
df.head()

Total de letras em inglês: 332731


,title,tag,artist,year,views,lyrics,language,len_lyrics
0,Ducky,rap,North Star (USA),2004,143,oh youll love repeats throughout intro christ ...,en,465
1,The Heist Revisited,rap,Big L,2000,5204,big yeah yeah yeahyeah yeahyeahyeahyeah yeah y...,en,696
2,Chinese New Year,rap,Clipse,2006,22502,chorus pusha pharrell im door eyes like judgin...,en,691
3,Pop That Cannon,rap,Cassidy,2004,677,intro swizz beatz aooow banger lets go swizz m...,en,562
4,Been This Way,rap,Scribe,2003,1014,intro every mc industry wants one thing best t...,en,785


# Tarefa (b): Predizer Gênero Musical

Objetivo: treinar modelos para prever o gênero de uma música com base no conteúdo da letra em inglês

## Sem Balanceamento

In [ ]:
# Separar dados
X = df['lyrics']
y = df['tag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [ ]:
# Vetorização com TF-IDF
tfidf = TfidfVectorizer()

# Modelos
models = {
  "Logistic Regression": LogisticRegression(max_iter=1000),
  "Linear SVC": LinearSVC()
}

for name, model in models.items():
  pipe = Pipeline([
    ('tfidf', tfidf),
    ('clf', model)
  ])
  print(f"\nTreinando modelo TF-IDF + {name}...")
  pipe.fit(X_train, y_train)
  preds = pipe.predict(X_test)

  print(f"\n== {name} com TF-IDF ==")
  print("Accuracy:", accuracy_score(y_test, preds))
  print("F1-score (macro):", f1_score(y_test, preds, average='macro'))
  print("Relatório de classificação:\n", classification_report(y_test, preds))


Treinando modelo TF-IDF + Logistic Regression...

== Logistic Regression com TF-IDF ==
Accuracy: 0.6795331596874374
F1-score (macro): 0.47995653951378675
Relatório de classificação:
               precision    recall  f1-score   support

     country       0.60      0.14      0.22      2550
        misc       0.85      0.38      0.53      3035
         pop       0.62      0.84      0.71     41767
         rap       0.86      0.86      0.86     28892
          rb       0.43      0.10      0.16      4679
        rock       0.54      0.32      0.40     18897

    accuracy                           0.68     99820
   macro avg       0.65      0.44      0.48     99820
weighted avg       0.67      0.68      0.65     99820


Treinando modelo TF-IDF + Linear SVC...

== Linear SVC com TF-IDF ==
Accuracy: 0.6712181927469445
F1-score (macro): 0.4716555568628625
Relatório de classificação:
               precision    recall  f1-score   support

     country       0.53      0.14      0.22      2550

## Com Balanceamento

In [ ]:
X_vec = TfidfVectorizer().fit_transform(df['lyrics'])
y_vec = df['tag']

print('Antes do balanceamento:', Counter(y_vec))

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_vec, y_vec)
print('Depois do balanceamento:', Counter(y_resampled))

Antes do balanceamento: Counter({'pop': 139223, 'rap': 96306, 'rock': 62990, 'rb': 15595, 'misc': 10117, 'country': 8500})
Depois do balanceamento: Counter({'rap': 139223, 'rb': 139223, 'rock': 139223, 'misc': 139223, 'country': 139223, 'pop': 139223})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled, random_state=42)

In [ ]:
# Modelos
models = {
  "Logistic Regression": LogisticRegression(max_iter=1000),
  "Linear SVC": LinearSVC()
}

for name, model in models.items():
  pipe = Pipeline([
    ('clf', model)
  ])
  print(f"\nTreinando modelo TF-IDF + {name}...")
  pipe.fit(X_train, y_train)
  preds = pipe.predict(X_test)

  print(f"\n== {name} com TF-IDF ==")
  print("Accuracy:", accuracy_score(y_test, preds))
  print("F1-score (macro):", f1_score(y_test, preds, average='macro'))
  print("Relatório de classificação:\n", classification_report(y_test, preds))


Treinando modelo TF-IDF + Logistic Regression...

== Logistic Regression com TF-IDF ==
Accuracy: 0.7419972705724616
F1-score (macro): 0.7348457521201389
Relatório de classificação:
               precision    recall  f1-score   support

     country       0.80      0.91      0.85     41767
        misc       0.84      0.88      0.86     41767
         pop       0.52      0.40      0.45     41767
         rap       0.87      0.83      0.85     41767
          rb       0.74      0.81      0.77     41767
        rock       0.63      0.63      0.63     41767

    accuracy                           0.74    250602
   macro avg       0.73      0.74      0.73    250602
weighted avg       0.73      0.74      0.73    250602


Treinando modelo TF-IDF + Linear SVC...

== Linear SVC com TF-IDF ==
Accuracy: 0.804690305743769
F1-score (macro): 0.7923043385548209
Relatório de classificação:
               precision    recall  f1-score   support

     country       0.84      0.97      0.90     41767
 